## 배열

In [17]:
%%writefile ../src/ArrayTest2.sol
pragma solidity ^0.6.0;
pragma experimental ABIEncoderV2;

contract ArrayTest2 {
    string[3] cities1=["Seoul", "Sydney", "Tokyo"];
    string[] cities2 = new string[](2);
    string[] cities3;
    int[5] mathMarks;
    uint[3][] marks=[[100, 80, 95],[20,30,40]];
    
    function getDynamicArrMemory() pure public returns(uint[] memory){
        uint[] memory num = new uint[](3);
        for(uint i = 0; i<num.length; i++)
            num[i] = i;
        return num;
    }
    
    function getStringDynamicArrMemory() pure public {
        string[2] memory places = ["9000", "Sydney"];
        //return places
        places[0] = "Seoul";
    }
    
    function getCities1() view public returns(string memory) {
        return cities1[0];
    }
    
    function setCities23() public {
        cities2[0]="New York";
        cities2.push("Busan");
        cities3.push("New York");
        cities3.push("Beijing");
    }
    
        //dynamic array return needs "pragma experimental ABIEncoderV2;"
    function getCities2() view public returns(string[] memory){
        return cities2;
    }
    
    function setMathMarks() public {
        mathMarks=[100,60,95,50,80];
    }
    
    function getMathAbove70() view public returns(int[] memory) {
        uint lengthOfMathAbove70=0;
        uint counter=0;
        
        for(uint8 i=0; i<mathMarks.length; i++)
            if(mathMarks[i]>70)
                lengthOfMathAbove70++;
        
        int[] memory mathAbove70=new int[](lengthOfMathAbove70);
        
        counter=0;
        for(uint i=0;i<mathMarks.length;i++) {
            if(mathMarks[i]>70) {
                mathAbove70[counter]=mathMarks[i];
                counter++;
            }
        }
        return mathAbove70;
    }
    
    function updateMarks() public returns(uint[3][] memory){
        marks[0][0]=90;
        return marks;
    }
    
    function getMarksArr() view public returns(uint[3][] memory) {
        return marks;
    }
    function getMarksLength() view public returns(uint) {
        return marks.length;
    }
}

Overwriting ../src/ArrayTest2.sol


In [25]:
!solc ../src/ArrayTest2.sol --combined-json abi > ..\src\ArrayTest2ABI.json

In [19]:
!solc ../src/ArrayTest2.sol --combined-json bin > ..\src\ArrayTest2BIN.json

In [20]:
%%writefile ../src/ArrayTest2Deploy.js
var Web3=require('web3');
var _abiJson = require('./ArrayTest2ABI.json');
var _binJson = require('./ArrayTest2BIN.json');

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
contractName=Object.keys(_abiJson.contracts); // reading ['src/ArrayTest2.sol:ArrayTest2']

_abiArray=JSON.parse(_abiJson.contracts[contractName].abi);    //JSON parsing needed!!
_bin=_binJson.contracts[contractName].bin; //ok without "0x"

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);

    new web3.eth.Contract(_abiArray).deploy({data: _bin}).estimateGas(function(err, gas) {
        if(!err) console.log(">> gas: "+ gas);
    });

    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1226648}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
    console.log("---> The contract deployed to: " + deployed.options.address)
}

deploy()

Overwriting ../src/ArrayTest2Deploy.js


In [21]:
!node ../src/ArrayTest2Deploy.js

Deploying the contract from 0xc5a8d5E52E0A97Bd23EbA81Fe82dF07858752fa9
>> gas: 1152955
hash: 0x8eb7a8dbb80da9b5b79ab6a3b5f11a256b8bc0102311419dd32880e149f9129b
---> The contract deployed to: 0x832dfE403e05a875740E3d77239791AC953eAEdA


In [22]:
%%writefile ../src/ArrayTest2Use.js
var Web3=require('web3');
var _abiJson = require('./ArrayTest2ABI.json');
var web3=new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));       //ok

contractName=Object.keys(_abiJson.contracts); // reading ['src/ArrayTest2.sol:ArrayTest2']
console.log("- contract name: ", contractName[0]); //or console.log(contractName);
_abiArray=JSON.parse(_abiJson.contracts[contractName].abi);    //JSON parsing needed!!

async function doIt() {
    var arr = new web3.eth.Contract(_abiArray, "0x832dfE403e05a875740E3d77239791AC953eAEdA");
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    
    arr.methods.setCities23().estimateGas(function(err, gas) {
        if(!err) console.log(">> gas: "+ gas);
    });

    //await arr.methods.setCities23().send({from: accounts[0]});   //out of gas error
    await arr.methods.setCities23().send({from: accounts[0], gas:230312});
    arr.methods.getCities2().call().then(console.log);

    arr.methods.setMathMarks().estimateGas(function(err, gas) {
        if(!err) console.log(">> gas: "+ gas);
    });

    await arr.methods.setMathMarks().send({from: accounts[0], gas: 122209});
    //ERROR invalid opcode arr.methods.getMathAbove70().call().then(console.log);
    arr.methods.getMarksArr().call().then(console.log);

    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    
}

doIt()

Overwriting ../src/ArrayTest2Use.js


In [23]:
!node ../src/ArrayTest2Use.js

- contract name:  ../src/ArrayTest2.sol:ArrayTest2
Account: 0xc5a8d5E52E0A97Bd23EbA81Fe82dF07858752fa9
Balance before: 99918399760000000000
>> gas: 135460
[ 'New York', '', 'Busan' ]
>> gas: 121979
[ [ '100', '80', '95' ], [ '20', '30', '40' ] ]
Balance after: 99913250980000000000
Balance diff: 5148779999985664


## 실습: Members 배열

In [27]:
%%writefile ../src/Members.sol
pragma solidity ^0.5.0;
contract Members{
    address owner;
    event printAddr(address arg);
    struct Member{
        uint id;
        string name;
    }
    Member[] public memberArr;
    constructor() public {
        owner=msg.sender;
    }
    function del() public {
        delete memberArr;
    }
    function delOne(uint i) public{
        delete memberArr[i];  //try pop()
    }
    function add(uint id,string memory name) public {
        memberArr.push(Member(id,name));
    }
    //return Member
    function getMember(string memory who) view public returns(uint, string memory) {
        uint _id;
        string memory _name;
        for(uint i=0;i<memberArr.length;i++) {
            _name=memberArr[i].name;
            if(keccak256(abi.encodePacked(_name))==keccak256(abi.encodePacked(who))) {
                _id=memberArr[i].id;
                _name=memberArr[i].name;
            }
        }
        return (_id,_name);
    }
    function compareStr(string memory s1, string memory s2) pure public returns(bool) {
        return keccak256(abi.encodePacked(s1))==keccak256(abi.encodePacked(s2));
    }
    function compareBytes(bytes memory b1, bytes memory b2) pure public returns(bool) {
        return keccak256(b1) == keccak256(b2);
    }
    function getLength() view public returns(uint) {
        return memberArr.length;
    }
}

Writing ../src/Members.sol


## 매핑